In [1]:
import os
import torch
import torchvision
from torchvision import transforms
from PIL import Image
import cv2
import numpy as np
import json

model_path = r'C:\Users\Admin\Downloads\final_model.pth'
class_mapping_path = r'D:\xla v1\model_output\class_mapping\class_mapping_V1.json'
image_path = r'D:\xla v1\input\images_test\h2.png'
output_image_path = r'D:\xla v1\model_output\images\output_image.jpg'

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

with open(class_mapping_path, 'r') as f:
    class_info = json.load(f)
    class_to_idx = class_info['class_to_idx']
    idx_to_class = {int(k): v for k, v in class_info['idx_to_class'].items()}
    num_classes = class_info['num_classes']

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, num_classes=num_classes+1)
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

transform = transforms.Compose([
    transforms.ToTensor()
])

def visualize_prediction(image_path, boxes, labels, scores, class_names, output_path, score_threshold=0.5):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    for i, box in enumerate(boxes):
        if scores[i] > score_threshold:
            xmin, ymin, xmax, ymax = map(int, box)
            class_idx = labels[i]
            class_name = class_names[class_idx - 1]
            confidence = scores[i]

            label = f"{class_name}: {confidence:.2f}"

            cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
            cv2.putText(image, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_path, image)
    print(f"Ảnh đã được lưu tại {output_path}")

def predict_image(image_path, model, transform, class_to_idx, idx_to_class, device, score_threshold=0.5):
    try:
        image = Image.open(image_path).convert("RGB")
        image_tensor = transform(image).to(device)
        image_tensor = image_tensor.unsqueeze(0)

        with torch.no_grad():
            prediction = model(image_tensor)

        boxes = prediction[0]['boxes'].cpu().numpy()
        labels = prediction[0]['labels'].cpu().numpy()
        scores = prediction[0]['scores'].cpu().numpy()

        class_names = [idx_to_class[label] for label in labels]

        filtered_boxes = boxes[scores > score_threshold]
        filtered_labels = labels[scores > score_threshold]
        filtered_scores = scores[scores > score_threshold]
        filtered_class_names = [idx_to_class[label] for label in filtered_labels]

        print("Kết quả dự đoán:")
        for i in range(len(filtered_boxes)):
            print(f"  - {filtered_class_names[i]}: {filtered_scores[i]:.4f}")

        visualize_prediction(image_path, filtered_boxes, filtered_labels, filtered_scores, list(idx_to_class.values()), output_image_path, score_threshold)

    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy ảnh tại đường dẫn {image_path}")
    except Exception as e:
        print(f"Lỗi trong quá trình nhận diện: {e}")

if __name__ == "__main__":
    predict_image(image_path, model, transform, class_to_idx, idx_to_class, device)


c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Kết quả dự đoán:
  - regulatory--no-entry: 0.9991
  - regulatory--keep-right: 0.9968
  - regulatory--no-entry: 0.9859
  - regulatory--no-entry: 0.9671
  - regulatory--no-right-turn: 0.9168
  - regulatory--maximum-speed-limit: 0.8555
  - regulatory--no-right-turn: 0.8284
  - regulatory--no-right-turn: 0.8056
  - regulatory--turn-left: 0.7389
  - regulatory--no-right-turn: 0.7335
  - regulatory--maximum-speed-limit: 0.6857
  - regulatory--no-heavy-goods-vehicles: 0.6094
Ảnh đã được lưu tại D:\xla v1\model_output\images\output_image.jpg
